In [1]:
# Dependencies
import pandas as pd
import numpy as np
import csv
import os

In [7]:
# File to Load (Remember to Change These)
file_to_load = "resources/crime-data_crime-data_crimestat.csv"
fziplist = r"resources/zipcodes.csv"
file_clean = "resources/clean_crime_data_with_rates.csv"
file_irs = "resources/clean_income_file.csv"

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(file_to_load)

df.head()

# df.count()

C:\Users\MariO\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE
0,201700001722914,11/01/2015 00:00,NaN,LARCENY-THEFT,279XX N 23RD LN,85085.0,SINGLE FAMILY HOUSE
1,201700001603695,11/01/2015 00:00,03/31/2016 00:00,RAPE,38XX W CAMELBACK RD,85019.0,HOSPITAL
2,201600000052855,11/01/2015 00:00,01/09/2016 00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029.0,SINGLE FAMILY HOUSE
3,201500002102668,11/01/2015 00:00,11/01/2015 11:50,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043.0,SINGLE FAMILY HOUSE
4,201500002168686,11/01/2015 00:00,11/11/2015 09:30,LARCENY-THEFT,14XX E HIGHLAND AVE,85014.0,PARKING LOT


In [4]:
# select only the interesting columns
crime_df = df.loc[:, ["ZIP", "OCCURRED ON"]]
crime_df.head()

,ZIP,OCCURRED ON
0,85085.0,11/01/2015 00:00
1,85019.0,11/01/2015 00:00
2,85029.0,11/01/2015 00:00
3,85043.0,11/01/2015 00:00
4,85014.0,11/01/2015 00:00


In [5]:
nb_crimes_df = crime_df.groupby("ZIP").count().reset_index()

# renaming the columns of nb of crimes
nb_crimes_df = nb_crimes_df.rename(columns={"OCCURRED ON":"nb of crimes",
                                           "ZIP":"ZipCode"})
nb_crimes_df["ZipCode"] = nb_crimes_df["ZipCode"].astype(int)

# .astype(str)
nb_crimes_df.head()



,ZipCode,nb of crimes
0,85003,2203
1,85004,2310
2,85006,3769
3,85007,3002
4,85008,8935


In [6]:
# Let's merge the data with the zipcode list we have --> so we have NaN values on the empty zipcodes

zip_df = pd.read_csv(fziplist, names=["ZipCode","City"])
# selected_zipcodes = list(zip_df["ZipCode"])
zip_df.head()
print(nb_crimes_df.count())

nb_crimes_df = pd.merge(zip_df, nb_crimes_df, on="ZipCode", how="left")

print(nb_crimes_df.count())



ZipCode         104
nb of crimes    104
dtype: int64
ZipCode         155
City            155
nb of crimes    103
dtype: int64


In [22]:
# to get an idea of the crime ratio per capita, we can load the income values from IRS that have the number of tax report
irs_df = pd.read_csv(file_irs)
# select the columns
irs_df = irs_df.loc[:, ["Zip Code", "Number of returns"]]
# rename the Zip Code column
irs_df = irs_df.rename(columns={ "Zip Code" : "ZipCode"})
# we need to remove the coma to be able to cast as number
irs_df['Number of returns'].replace(regex=True,inplace=True,to_replace=r',',value=r'')
irs_df = irs_df.dropna()
irs_df["Number of returns"] = irs_df["Number of returns"].astype(int)
irs_df.head()

,ZipCode,Number of returns
0,85003,3900
1,85004,2960
2,85006,9680
3,85007,5300
4,85008,23140


In [27]:
# let's merge the dataframes
nb_crimes_df2 = pd.merge(nb_crimes_df, irs_df, on="ZipCode", how="left")

# let's calculate the ratio
nb_crimes_df2['crime per tax payer'] = nb_crimes_df2['nb of crimes']/nb_crimes_df2['Number of returns']

nb_crimes_df2.head()

,ZipCode,City,nb of crimes,Number of returns,crime per tax payer
0,85003,Phoenix,2203.0,3900.0,0.564872
1,85004,Phoenix,2310.0,2960.0,0.780405
2,85006,Phoenix,3769.0,9680.0,0.389360
3,85007,Phoenix,3002.0,5300.0,0.566415
4,85008,Phoenix,8935.0,23140.0,0.386128


In [14]:
# then save it to a csv file
nb_crimes_df.to_csv(file_clean, index=False)